In [1]:
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
import requests
import json
import unicodedata
from difflib import SequenceMatcher

In [19]:
def similarity_ratio(str1, str2):
    """Calculates the similarity ratio between two strings."""
    return SequenceMatcher(None, str1, str2).ratio()

def find_book_google_books_advanced(title, author, title_similarity_threshold=0.8, author_similarity_threshold=0.8):
    """
    Searches the Google Books API for a book using advanced matching logic.

    Args:
        title (str): The title of the book.
        author (str): The author of the book.
        title_similarity_threshold (float): Minimum similarity ratio for title match (0.0 to 1.0).
        author_similarity_threshold (float): Minimum similarity ratio for author match (0.0 to 1.0).

    Returns:
        dict or None: A dictionary containing information about the best matching book,
                     or None if no sufficiently similar book is found or an error occurs.
    """
    base_url = "https://www.googleapis.com/books/v1/volumes"
    params = {"q": f"{title} inauthor:{author}"}
    best_match = None
    highest_similarity = 0.0

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        items = data.get("items", [])

        if items:
            for item in items:
                info = item.get("volumeInfo", {})
                title_api = info.get("title", "").lower()
                authors_api = [a.lower() for a in info.get("authors", [])]

                title_similarity = similarity_ratio(title.lower(), title_api)
                author_similarity = 0.0
                if authors_api:
                    max_author_similarity = 0.0
                    for author_from_api in authors_api:
                        current_similarity = similarity_ratio(author.lower(), author_from_api)
                        max_author_similarity = max(max_author_similarity, current_similarity)
                    author_similarity = max_author_similarity
                elif not author:  # If no author is provided in the search, consider it a match
                    author_similarity = 1.0

                combined_similarity = (title_similarity + author_similarity) / 2.0

                if title_similarity >= title_similarity_threshold and author_similarity >= author_similarity_threshold:
                    if combined_similarity > highest_similarity:
                        highest_similarity = combined_similarity
                        best_match = info

        # If no good match with both title and author, try searching by title only with a lower threshold
        if not best_match:
            params_title_only = {"q": title}
            response_title_only = requests.get(base_url, params=params_title_only)
            response_title_only.raise_for_status()
            data_title_only = response_title_only.json()
            items_title_only = data_title_only.get("items", [])

            if items_title_only:
                for item in items_title_only:
                    info = item.get("volumeInfo", {})
                    title_api = info.get("title", "").lower()
                    title_similarity = similarity_ratio(title.lower(), title_api)
                    if title_similarity >= title_similarity_threshold - 0.1: # Lower threshold for title-only search
                        if title_similarity > highest_similarity:
                            highest_similarity = title_similarity
                            best_match = info

        return best_match

    except requests.exceptions.RequestException as e:
        print(f"Error during API request: {e}")
        return None
    
def add_columns(sheet, num_columns_to_add):
    """Adds a specified number of columns to the Google Sheet."""
    try:
        properties = sheet.fetch_sheet_metadata()['sheets'][0]['properties']
        current_column_count = properties['gridProperties']['columnCount']
        new_column_count = current_column_count + num_columns_to_add

        body = {
            "requests": [
                {
                    "updateSheetProperties": {
                        "properties": {
                            "gridProperties": {
                                "columnCount": new_column_count
                            }
                        },
                        "fields": "gridProperties.columnCount"
                    }
                }
            ]
        }
        sheet.batch_update(body)

    except gspread.exceptions.APIError as e:
        print(f"Error adding columns: {e}")

def insert_info_in_sheet(sheet, info_to_add):
    """
    Inserts information into the Google Sheets based on the provided parameters.

    Args:
        sheet (gspread.models.Spreadsheet): The Google Sheets object.
        info_to_add (str): The field name (e.g. 'Category') to be fetched and added to the sheet.

    Returns:
        None
    """
    sheet1 = sheet.sheet1
    values_list = sheet1.get_all_values()

    info_searched = info_to_add.lower()
    header_row = values_list[0]
    num_cols = len(header_row)

    # Get column indices for title and author
    title_column_index = header_row.index("Book_Title") if "Book_Title" in header_row else 0
    author_column_index = header_row.index("Author") if "Author" in header_row else 1

    # Add new column if info_to_add is not in header
    if info_searched.capitalize() not in header_row:
        sheet1.add_cols(1)
        num_cols += 1
        sheet1.update_cell(1, num_cols, info_searched.capitalize())
        info_column_index = num_cols - 1
    else:
        info_column_index = header_row.index(info_searched.capitalize())

    # Get rows where info_to_add is empty or missing
    empty_rows = [
        i + 2 for i, row in enumerate(values_list[1:])  # skip header
        if len(row) <= info_column_index or not row[info_column_index]
    ]

    try:
        for row_index in empty_rows:
            if row_index - 1 < len(values_list):
                book = values_list[row_index - 1]
                if len(book) <= max(title_column_index, author_column_index):
                    continue  # skip incomplete rows

                title_name = book[title_column_index].strip()
                author_name = book[author_column_index].strip()

                book_api_info = find_book_google_books_advanced(title_name, author_name)

                if book_api_info and info_to_add in book_api_info:
                    info_value = book_api_info[info_to_add]
                    if not info_value:
                        info_value = "Unknown"
                else:
                    info_value = "Unknown"

                print(f"Updating row {row_index}: '{title_name}' → {info_searched.capitalize()} = {info_value}")
                # Convert list or other non-string types to string
                if isinstance(info_value, list):
                    info_value = ", ".join(info_value)
                elif not isinstance(info_value, str):
                    info_value = str(info_value)

                sheet1.update_cell(row_index, info_column_index + 1, info_value)


    except gspread.exceptions.APIError as e:
        print(f"Google Sheets API Error: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def get_book_keys(sheet):
    """
    Efficiently retrieves the set of keys returned by the Google Books API for each unique book.
    Avoids redundant API calls and skips header row properly.

    Args:
        sheet (gspread.models.Spreadsheet): The Google Sheets object.

    Returns:
        list: A list of unique sets of keys from the book API results.
    """
    values_list = sheet.sheet1.get_all_values()
    header_row = values_list[0]

    try:
        title_col = header_row.index("Book_Title")
    except ValueError:
        print("Warning: 'Book_Title' column not found. Assuming index 0.")
        title_col = 0

    try:
        author_col = header_row.index("Author")
    except ValueError:
        print("Warning: 'Author' column not found. Assuming index 1.")
        author_col = 1

    keys = []
    seen_titles = set()

    for row in values_list[1:]:  # skip header
        if len(row) <= max(title_col, author_col):
            continue  # skip malformed rows

        title = row[title_col].strip()
        author = row[author_col].strip()

        key = (title.lower(), author.lower())
        if key in seen_titles:
            continue  # skip duplicates

        seen_titles.add(key)

        book_info = find_book_google_books_advanced(title, author)
        if book_info:
            api_keys_set = frozenset(book_info.keys())
            if api_keys_set not in keys:
                keys.append(api_keys_set)

    return keys

def save_sheet_as_sql(sheet, table_name="books", output_file="sheet_export.sql"):
    """
    Saves the contents of a Google Sheet as SQL INSERT statements in a .sql file.

    Args:
        sheet (gspread.models.Spreadsheet): Authenticated Google Sheet object.
        table_name (str): Name of the SQL table.
        output_file (str): Path to the output .sql file.
    """
    sheet1 = sheet.sheet1
    values = sheet1.get_all_values()

    headers = values[0]  # column names
    rows = values[1:]    # data rows

    with open(output_file, "w", encoding="utf-8") as f:
        for row in rows:
            # Pad row to match header length
            row += [""] * (len(headers) - len(row))

            # Sanitize and format values
            sanitized_values = []
            for cell in row:
                # Handle strings and escape single quotes
                if isinstance(cell, str):
                    cell = cell.replace("'", "''")
                    sanitized_values.append(f"'{cell}'")
                else:
                    sanitized_values.append("NULL" if cell == "" else str(cell))

            insert_stmt = f"INSERT INTO {table_name} ({', '.join(headers)}) VALUES ({', '.join(sanitized_values)});"
            f.write(insert_stmt + "\n")

    print(f"SQL export complete: {output_file}")

def save_sheet_as_json(sheet, output_filename):
    """
    Save the contents of a Google Sheet as a JSON file.
    Args:
        sheet (gspread.models.Spreadsheet): Authenticated Google Sheet object.
        output_filename (str): Path to the output JSON file (without extension).  
    """

    try:
        # Get all values from the worksheet as a list of lists
        data = sheet.sheet1.get_all_values()

        # Convert the list of lists into a list of dictionaries (assuming the first row is the header)
        if data:
            headers = data[0]
            json_data = [dict(zip(headers, row)) for row in data[1:]]
        else:
            json_data = []

        # Save the JSON data to a file
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=4, ensure_ascii=False)

        print(f"Sheet '{sheet}' saved as '{output_filename}'")
    
    except Exception as e:
        print(f"Error saving sheet as JSON: {e}")

def set_first_row_filter(sheet):
    """
    Sets a filter on the first row of the first worksheet in the given Google Sheet.

    Args:
        sheet (gspread.models.Spreadsheet): The Google Sheets object.

    Returns:
        None
    """
    sheet1 = sheet.sheet1
    values_list = sheet1.get_all_values()
    if not values_list:
        print("The sheet is empty, cannot set a filter.")
        return

    num_cols = len(values_list[0])
    if num_cols == 0:
        print("The first row is empty, cannot set a filter.")
        return

    try:
        end_column_letter = chr(ord('A') + num_cols - 1)
        filter_range = f"{sheet1.title}!A1:{end_column_letter}1"
        sheet.add_filter(filter_range)
        print(f"Filter applied to the first row of '{sheet1.title}'.")
    except gspread.exceptions.APIError as e:
        print(f"Error applying filter: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [4]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
creds = Credentials.from_service_account_file("credentials.json", scopes=SCOPES)
gc = gspread.authorize(creds)

In [5]:
sheet_id = "19JV7gS3Iy_MTOluozSech3iFmkxRF11oCVYLhp1ziN0"
sheet = gc.open_by_key(sheet_id)

values_list = sheet.sheet1.get_all_values()
records = sheet.sheet1.get_all_records()  # fast and efficient

In [18]:
pd.DataFrame(values_list, columns=values_list[0]).drop(index=0)

,Book_Title,Author,Publisher,Number_of_pages,Translator,Year_bought,Status,Owner
1,O Cortiço,Aluísio Azevedo,Sextante,110,-,1976,Não Lido,Diego
2,Harry Potter e a Pedra Filosofal,J. K. Rowling,Rocco,263,Lia Wyler,2000,Lido,Hugo
3,Harry Potter e a Câmara Secreta,J. K. Rowling,Rocco,287,Lia Wyler,2001,Lido,Hugo e Diego
4,Harry Potter e o Cálice de Fogo,J. K. Rowling,Rocco,583,Lia Wyler,2002,Lido,Hugo e Diego
5,Harry Potter e o Prizioneiro de Azkaban,J. K. Rowling,Rocco,348,Lia Wyler,2002,Lido,Diego
...,...,...,...,...,...,...,...,...
106,O Senhor do Arco,Conn Iggulden,Record,418,Alves Calado,-,Não Lido,Diego
107,Quadribol através dos séculos,J. K. Rowling,Rocco,63,Lia Wyler,-,Lido,Diego
108,Sherlock Holme - O Cão de Baskervilles,Sr. Arthur Connan Doyle,Melhoramentos,336,Antonio Carlos Vilela,-,Lido,Diego
109,Sherlock Holmes - O Vale do Terror,Sr. Arthur Connan Doyle,Melhoramentos,326,Antonio Carlos Vilela,-,Lido,Diego


In [6]:
library = pd.DataFrame(values_list, columns=values_list[0]).drop(index=0)
library["Status"].unique()  # Check unique values in the Status column

array(['Não Lido', 'Lido'], dtype=object)

In [20]:
infos_to_search = get_book_keys(sheet)
print(f"Infos to search: {infos_to_search}")

Infos to search: [frozenset({'categories', 'description', 'allowAnonLogging', 'panelizationSummary', 'pageCount', 'printType', 'readingModes', 'canonicalVolumeLink', 'industryIdentifiers', 'publishedDate', 'contentVersion', 'infoLink', 'maturityRating', 'authors', 'publisher', 'title', 'imageLinks', 'language', 'previewLink'}), frozenset({'categories', 'title', 'publishedDate', 'description', 'pageCount', 'contentVersion', 'infoLink', 'previewLink', 'maturityRating', 'printType', 'language', 'readingModes', 'authors', 'allowAnonLogging', 'panelizationSummary', 'canonicalVolumeLink', 'industryIdentifiers'}), frozenset({'categories', 'description', 'ratingsCount', 'allowAnonLogging', 'panelizationSummary', 'pageCount', 'printType', 'readingModes', 'canonicalVolumeLink', 'industryIdentifiers', 'publishedDate', 'contentVersion', 'infoLink', 'maturityRating', 'authors', 'averageRating', 'publisher', 'title', 'imageLinks', 'language', 'previewLink'}), frozenset({'categories', 'title', 'publi

In [18]:
info = "categories"

insert_info_in_sheet(sheet, info)

Updating row 2: 'O Cortiço' → Categories = ['Young Adult Fiction']
Updating row 3: 'Harry Potter e a Pedra Filosofal' → Categories = ['Fiction']
Updating row 4: 'Harry Potter e a Câmara Secreta' → Categories = ['Juvenile Fiction']
Updating row 5: 'Harry Potter e o Cálice de Fogo' → Categories = ['Adventure stories']
Updating row 6: 'Harry Potter e o Prizioneiro de Azkaban' → Categories = ['Adventure stories']
Updating row 7: 'Meu Pé de Laranja Lima' → Categories = Unknown
Updating row 8: 'Amor não tem cor' → Categories = Unknown
Updating row 9: 'Auto da Compadecida' → Categories = ['Drama']
Updating row 10: 'Memórias póstumas de Brás Cubas' → Categories = ['Fiction']
Updating row 11: 'O Fantasma de Canterville - Uma novela e três contos' → Categories = ['Fiction']
Updating row 12: 'O Grande Mentecapto' → Categories = ['Fiction']
Updating row 13: 'Ponciá Vicêncio' → Categories = ['Fiction']
Updating row 14: 'Senhora' → Categories = Unknown
Updating row 15: '26 Poetas Hoje' → Categories 

In [27]:
info = "language"

insert_info_in_sheet(sheet, info)

Updating row 2: 'O Cortiço' → Language = pt-BR
Updating row 3: 'Harry Potter e a Pedra Filosofal' → Language = pt-BR
Updating row 4: 'Harry Potter e a Câmara Secreta' → Language = pt-BR
Updating row 5: 'Harry Potter e o Cálice de Fogo' → Language = pt-BR
Updating row 6: 'Harry Potter e o Prizioneiro de Azkaban' → Language = pt-BR
Updating row 7: 'Meu Pé de Laranja Lima' → Language = pt-BR
Updating row 8: 'Amor não tem cor' → Language = pt-BR
Updating row 9: 'Auto da Compadecida' → Language = pt-BR
Updating row 10: 'Memórias póstumas de Brás Cubas' → Language = pt
Updating row 11: 'O Fantasma de Canterville - Uma novela e três contos' → Language = pt-BR
Updating row 12: 'O Grande Mentecapto' → Language = pt-BR
Updating row 13: 'Ponciá Vicêncio' → Language = pt-BR
Updating row 14: 'Senhora' → Language = pt-BR
Updating row 15: '26 Poetas Hoje' → Language = pt-BR
Updating row 16: 'Maira' → Language = en
Updating row 17: 'O recado do Morro' → Language = pt-BR
Updating row 18: 'Prosas seguid

In [28]:
info = "publishedDate"

insert_info_in_sheet(sheet, info)

Updating row 2: 'O Cortiço' → Publisheddate = 2024-07-20
Updating row 3: 'Harry Potter e a Pedra Filosofal' → Publisheddate = 2023-01-17
Updating row 4: 'Harry Potter e a Câmara Secreta' → Publisheddate = 2015-12-08
Updating row 5: 'Harry Potter e o Cálice de Fogo' → Publisheddate = 2002
Updating row 6: 'Harry Potter e o Prizioneiro de Azkaban' → Publisheddate = 2002
Updating row 7: 'Meu Pé de Laranja Lima' → Publisheddate = 2013-03-26
Updating row 8: 'Amor não tem cor' → Publisheddate = 2002
Updating row 9: 'Auto da Compadecida' → Publisheddate = 2024-11-21
Updating row 10: 'Memórias póstumas de Brás Cubas' → Publisheddate = 2024-06-28
Updating row 11: 'O Fantasma de Canterville - Uma novela e três contos' → Publisheddate = 2021-07-30
Updating row 12: 'O Grande Mentecapto' → Publisheddate = 2020-06-29
Updating row 13: 'Ponciá Vicêncio' → Publisheddate = 2020-08-10
Updating row 14: 'Senhora' → Publisheddate = 2010
Updating row 15: '26 Poetas Hoje' → Publisheddate = 2021-02-24
Updating 

In [29]:
save_sheet_as_sql(sheet, table_name="books", output_file="MyLibrary.sql")

SQL export complete: MyLibrary.sql


In [30]:

save_sheet_as_json(sheet, output_filename= "MyLibrary.json")

Sheet '<Spreadsheet 'Biblioteca' id:19JV7gS3Iy_MTOluozSech3iFmkxRF11oCVYLhp1ziN0>' saved as 'MyLibrary.json'


___